# Get osm, project sidewalk and traffic sign crossings

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.geometry as sg
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely import wkt
import momepy
import networkx as nx
from scipy.spatial.distance import cdist
from branca.element import Template, MacroElement
from tqdm import tqdm

import folium

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Import walking network 

In [ ]:
# Get basic pedestrian network with widths
CRS = 'EPSG:28992'

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes.reset_index()
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')

In [ ]:
# Calculate connected components for pedestrian network
cc_network = list(nx.connected_components(graph_network))
nodes_in_edge = [line.coords[0] for line in gdf_network['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_network)) if node in cc_network[i]]
gdf_network_nodes['cc'] = [x for y in zip(cc_ids, cc_ids) for x in y]

# 1. Create crossings with OSM crossing features

### Import osm crossing features

In [ ]:
# # Import osm crossing features
osm_crossing_features = cf.in_folder + 'route_planning/crossing_features/OpenStreetMap/crossing_features_osm.csv'
df_osm_cf = pd.read_csv(osm_crossing_features)
df_osm_cf['geometry'] = df_osm_cf['geometry'].apply(wkt.loads)
gdf_osm_cf = gpd.GeoDataFrame(df_osm_cf, crs='EPSG:4326').to_crs(crs=CRS).reset_index()

### Select crossing features and nodes to keep


In [ ]:
# Create network and crossing feature dataframes
gdf_osm_cf_sel = gdf_osm_cf.loc[gdf_osm_cf['highway'] != 'cycleway']
gdf_osm_nodes = gdf_osm_cf_sel[gdf_osm_cf_sel['element_type'] == 'node']
gdf_osm_edges = gdf_osm_cf_sel[gdf_osm_cf_sel['element_type'] == 'way']

# Remove crossing nodes that already are on a crossing edge
for i, row in gdf_osm_nodes.iterrows():
    gdf_osm_nodes.loc[i, 'min_dis'] = gdf_osm_edges.distance(row['geometry']).min()
gdf_osm_nodes = gdf_osm_nodes.loc[gdf_osm_nodes['min_dis'] > 0.01]

### Initiate function to match connect nodes in the network

In [ ]:
# Function to find closest target nodes from source nodes and subsequently create edge between tese nodes.
# Edge is a connection between a source and a target node.
# max_dist: maximmum distance between source and target node is max_dist.
# max_connections: maximum number of new edges from source node.
# include_cc_rule: Restricts nodes to be connected to the same walking network component multiple times
def get_crossing_edges(gdf_source_nodes, gdf_target_nodes, max_dist=20, max_connections=3, CRS='EPSG:28992', include_cc_rule=False):

    # Get distance matrices
    dist_matrix = cdist(gdf_source_nodes[['x', 'y']].values, gdf_target_nodes[['x', 'y']].values, metric='euclidean')
    np.fill_diagonal(dist_matrix, 10**6)
    dist_sort = np.sort(dist_matrix, axis=1)
    dist_argsort = np.argsort(dist_matrix, axis=1)

    # Calculate new edges
    idxs = np.where(dist_sort < max_dist, True, False)
    nodes_to_connect = [list(row[row_idxs]) for row, row_idxs in zip(dist_argsort, idxs)]

    if include_cc_rule:
        edges_geometries = []
        for source_node in tqdm(range(len(dist_sort))):
            restricted_cc, connections_count = [], 0

            for target_node in nodes_to_connect[source_node]:
                target_cc = gdf_target_nodes.iloc[target_node]['cc']
                if target_cc not in restricted_cc and connections_count < max_connections:
                    edge = sg.LineString([gdf_source_nodes.iloc[source_node]['geometry'], gdf_target_nodes.iloc[target_node]['geometry']])
                    edges_geometries.append(edge)
                    restricted_cc.append(target_cc)
                    connections_count += 1

    else:
        edges = [[i, nodes_to_connect[i][j]] for i in range(len(dist_sort)) for j in range(len(nodes_to_connect[i])) if j < max_connections]
        edges_geometries = [sg.LineString([gdf_source_nodes.iloc[edge[0]]['geometry'], gdf_target_nodes.iloc[edge[1]]['geometry']]) for edge in edges]
    
    gdf_edges = gpd.GeoDataFrame(geometry=edges_geometries, crs=CRS)
    return gdf_edges

### Add osm crosswalk edges to the network

In [ ]:
# # Current heuristic: find closest node in the network and connect outer points of crosswalk edge to that point

# Get outer nodes of crossing edges and connect to network
gdf_osm_edge_nodes = gpd.GeoDataFrame(geometry=gdf_osm_edges['geometry'].boundary.explode(index_parts=True), crs=CRS).reset_index()
gdf_osm_edge_nodes['x'], gdf_osm_edge_nodes['y'] = gdf_osm_edge_nodes.geometry.x, gdf_osm_edge_nodes.geometry.y
gdf_new_edges_from_osm_edges = get_crossing_edges(gdf_osm_edge_nodes, gdf_network_nodes, max_dist=10, max_connections=2, include_cc_rule=True)
print('Number of new edges from crosswalk edge to network node:', len(gdf_new_edges_from_osm_edges))

### Add osm crosswalk nodes to the network

In [ ]:
# TODO
# Include number of intersections with road as selection criteria to keep or delete new edges
# Include length of subnetwork that is connected to crossing feature node by new edge as selection criteria to keep or delete new edges

# Get coordinates of crossing feature nodes
gdf_osm_nodes['x'], gdf_osm_nodes['y'] = gdf_osm_nodes.geometry.x, gdf_osm_nodes.geometry.y

# Connect osm crosswalk nodes to eachother
gdf_self_edges = get_crossing_edges(gdf_osm_nodes, gdf_osm_nodes, max_dist=10, max_connections=2, include_cc_rule=False)
print('Number of new edges from crossing node to crossing node:', len(gdf_self_edges))

# Connect osm crosswalk nodes to network
gdf_network_edges = get_crossing_edges(gdf_osm_nodes, gdf_network_nodes, max_dist=10, max_connections=3, include_cc_rule=True)
print('Number of new edges from osm node to network node:', len(gdf_network_edges))

### create map and visualize crossings on map

In [ ]:
# Initialize legend
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Network Edge </li>
    <li><span style='background:orange;opacity:0.7;'></span>Crossing Edge</li>
    <li><span style='background:red;opacity:0.7;'></span>Crossing Edge <-> Network Node</li>
    <li><span style='background:yellow;opacity:0.7;'></span>Crossing Node <-> Network Node</li>
    <li><span style='background:limegreen;opacity:0.7;'></span>Crossing Node <-> Crossing Node</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

In [ ]:
# Create tooltip for feature representation on map
def gen_tooltip(fields, aliases):

    tooltip = folium.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    return tooltip

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_osm_edges, style_function=lambda x: {"color": "orange", "weight": 4}).add_to(map)
geo_j = folium.GeoJson(gdf_new_edges_from_osm_edges, style_function=lambda x: {"color": "red", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_osm_cf_sel.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_osm_cf_sel, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Save map
map.get_root().add_child(macro)
map.save(cf.out_folder + 'extracted_crossings/osm_crossing_edges.html')

### Save new edges

In [ ]:
gdf_edges = pd.concat([gdf_osm_edges, gdf_new_edges_from_osm_edges, gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_osm_crossings, driver='GPKG')

## 2. Create crossings with Project Sidewalk crossing features

### Import Project sidewalk crossing features

In [ ]:
# Import project sidewalk crossing features
pj_crossing_features = cf.in_folder + 'route_planning/crossing_features/project_sidewalk/crossing_features_project_sidewalk.csv'
df_ps_cf = pd.read_csv(pj_crossing_features)
df_ps_cf['geometry'] = df_ps_cf['geometry'].apply(wkt.loads)
gdf_ps_cf = gpd.GeoDataFrame(df_ps_cf, crs=CRS).reset_index()

### Select crossing features and nodes to keep

In [ ]:
# Create network and crossing feature dataframes
gdf_ps_nodes = gdf_ps_cf.loc[gdf_ps_cf['label_type'] == 'Crosswalk']

# TODO Check if ['CurbRamp' 'Obstacle' 'Signal' 'NoCurbRamp' 'SurfaceProblem'] nodes are useful to find crossings as well

### Add project sidewalk nodes to network

In [ ]:
# TODO
# Include number of intersections with road as selection criteria to keep or delete new edges
# Include length of subnetwork that is connected to crossing feature node by new edge as selection criteria to keep or delete new edges

# Get coordinates of crossing feature and network nodes
gdf_ps_nodes['x'], gdf_ps_nodes['y'] = gdf_ps_nodes.geometry.x, gdf_ps_nodes.geometry.y

# Connect osm crosswalk nodes to eachother
gdf_self_edges = get_crossing_edges(gdf_ps_nodes, gdf_ps_nodes, max_dist=20, max_connections=2, include_cc_rule=False)
print('Number of new edges from crosswalk node to crosswalk node:', len(gdf_self_edges))

# Connect osm crosswalk nodes to network
gdf_network_edges = get_crossing_edges(gdf_ps_nodes, gdf_network_nodes, max_dist=10, max_connections=3, include_cc_rule=True)
print('Number of new edges from crosswalk node to network node:', len(gdf_network_edges))

### Create map and add crossings

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add network edges and crossing nodes
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_ps_nodes.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_ps_nodes, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Save map
map.get_root().add_child(macro)
map.save(cf.out_folder + 'extracted_crossings/project_sidwalk_crossing_edges.html')

### Save new edges

In [ ]:
gdf_edges = pd.concat([gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_project_sidewalk_crossings, driver='GPKG')

## 3. Add crossings with traffic sign features 

### Import traffic sign features

In [ ]:
# Import project traffic sign features
tf_features = cf.in_folder + 'route_planning/crossing_features/traffic_sign/crossing_features_traffic_sign.csv'
df_ts_f = pd.read_csv(tf_features)
df_ts_f['geometry'] = df_ts_f['geometry'].apply(wkt.loads)
gdf_ts_nodes = gpd.GeoDataFrame(df_ts_f, crs='EPSG:4326').to_crs(crs=CRS).reset_index()

### Add traffic sign nodes to network

In [ ]:
# TODO
# Include number of intersections with road as selection criteria to keep or delete new edges
# Include length of subnetwork that is connected to crossing feature node by new edge as selection criteria to keep or delete new edges

# Get coordinates of crossing feature and network nodes
gdf_ts_nodes['x'], gdf_ts_nodes['y'] = gdf_ts_nodes.geometry.x, gdf_ts_nodes.geometry.y

# Connect osm crosswalk nodes to eachother
gdf_self_edges = get_crossing_edges(gdf_ts_nodes, gdf_ts_nodes, max_dist=20, max_connections=2, include_cc_rule=False)
print('Number of new edges from traffic sign node to traffic sign node:', len(gdf_self_edges))

# Connect osm crosswalk nodes to network
gdf_network_edges = get_crossing_edges(gdf_ts_nodes, gdf_network_nodes, max_dist=12, max_connections=3, include_cc_rule=True)
print('Number of new edges from traffic sign node to network node:', len(gdf_network_edges))

### Create map and add crossings

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add network edges and crossing nodes
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_ts_nodes.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_ts_nodes, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Save map
map.get_root().add_child(macro)
map.save(cf.out_folder + 'extracted_crossings/traffic_sign_crossing_edges.html')

### Save new edges

In [ ]:
gdf_edges = pd.concat([gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_traffic_sign_crossings, driver='GPKG')